
## Highlighting the Contextual Issue

Let's first ask the model about a topic that it hasn't seen during pretraining, specifically, let's ask it who is teaching the currrent class.



In [ ]:
!pip install langchain_huggingface

In [ ]:
!pip install -U langchain langchain-mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 37.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.0.7
    Uninstalling langchain-core-1.0.7:
      Successfully uninstalled langchain-core-1.0.7
  Attempting uninstall: langchain
    Found existing installation: langchain 1.0.8
    Uninstalling langchain-1.0.8:
      Successfully uninstalled langchain-1.0.8


In [ ]:
!pip install langchain_classic
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 130.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 136.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 12.6 MB/s

In [ ]:
from langchain_mistralai import ChatMistralAI
import os
from google.colab import userdata

llm = ChatMistralAI(model="mistral-small",  mistral_api_key=userdata.get('MISTRAL_AI_KEY')) # Or other Mistral models

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful AI assistant."),
        ("user", "{input}")
    ])

chain = prompt | llm | StrOutputParser()

In [ ]:
response = chain.invoke({"input": "Who teaches the AIM-5011-1: Natural Language Processing, Fall 2025 course at Katz?"})
print(response)

I'm sorry for any confusion, but I don't have real-time or future data access to provide information about specific courses or instructors at the University of Pittsburgh's Katz Graduate School of Business for the Fall 2025 semester. Course offerings, instructors, and schedules are typically announced closer to the semester in question.

For the most accurate and up-to-date information, I would recommend checking the official Katz Graduate School of Business website or contacting their admissions or academic advising offices directly. They would be the best sources for details about the AIM-5011-1: Natural Language Processing course and its instructor for Fall 2025.


As expected, Mistral hasn't the slightest idea who's teaching the current class. We need to create a basic RAG system that will retrieve the right context to answer that question.

First, let's grab the syllabus from the local directory and do some basic chunking.

In [ ]:

from langchain_classic.document_loaders import TextLoader, DirectoryLoader

from pathlib import Path
docs_dir = "."
loader = DirectoryLoader(
    docs_dir, glob=f"syllabus.md", show_progress=True, loader_cls=TextLoader

)
data = loader.load()

100%|██████████| 1/1 [00:00<00:00, 3587.94it/s]


In [ ]:
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from tqdm import tqdm

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=200,
    length_function=len,

)
text_docs = text_splitter.split_documents(data)

Let's look at a few chunks

In [ ]:

def display_doc_split(docs):

    """display information about the split documents"""

    print("\n--- Document Chunks Information ---")
    print(f"Number of document chunks: {len(docs)}")
    print(f"Sample chunk:\n{docs[0].page_content}\n")


display_doc_split(text_docs)


--- Document Chunks Information ---
Number of document chunks: 18
Sample chunk:
# AIM-5011-1: Natural Language Processing, Fall 2025

> Repo for my Fall 2025 Natural Language Processing course, an elective in the M.S. Program in Artificial Intelligence, Katz School of Science and Health, Yeshiva University


Instructor: Adam Faulkner(adam.faulkner@yu.edu) \
Class hours: Tuesdays 5:30-7:30pm  \
Classroom: 205 Lexington Avenue Rm. 700 

## Course Description



Next, we need a way to embed and index the chunks. We'll use a `sentence-transformers` embedding model for this. The database we'll be using is `chromadb`

In [ ]:

from langchain_huggingface import HuggingFaceEmbeddings
# create embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
from langchain_classic.vectorstores import Chroma

persist_directory = "db"
vectordb = Chroma.from_documents(
    documents=text_docs, embedding=embeddings, persist_directory=persist_directory

)


In [ ]:
from langchain.tools import tool
@tool(response_format="content_and_artifact")
def retrieve_context(query: str, k=10):
    """Retrieve information to help answer a query."""
    retriever = vectordb.as_retriever(

    search_type="similarity",

    search_kwargs={"k":k},
    )
    relevant_docs = retriever.invoke(query)
    return relevant_docs
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in relevant_docs
    )
    return serialized, relevant_docs


In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
format_docs(text_docs)

"# AIM-5011-1: Natural Language Processing, Fall 2025\n\n> Repo for my Fall 2025 Natural Language Processing course, an elective in the M.S. Program in Artificial Intelligence, Katz School of Science and Health, Yeshiva University\n\n\nInstructor: Adam Faulkner(adam.faulkner@yu.edu) \\\nClass hours: Tuesdays 5:30-7:30pm  \\\nClassroom: 205 Lexington Avenue Rm. 700 \n\n## Course Description\n\nThis course provides a comprehensive overview of Natural Language Processing, the sub-field of Artificial Intelligence that deals with the automated processing of natural language text. The course is divided into three sections. In the first section, we'll introduce classic approaches to modeling natural language using traditional machine learning techniques such as Logistic Regression, Naive Bayes, and n-gram language modeling, as well as sparse vector-based representations of text such as\n\nnatural language using traditional machine learning techniques such as Logistic Regression, Naive Bayes, 

Now let's create a new prompt that forces the LLM to refer to the results of querying the RAG database given the prompt.

In [ ]:
# imports
from langchain_core.prompts import PromptTemplate

# Create the Prompt Template
prompt_template = """Use the context provided to answer
the user's question below. If you do not know the answer
based on the context provided, tell the user that you do
not know the answer to their question based on the context
provided and that you are sorry.

context: {context}

question: {query}

answer: """

# Create Prompt Instance from template
custom_rag_prompt = PromptTemplate.from_template(prompt_template)

In [ ]:
# imports
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Create the RAG Chain
rag_chain = (
    {"context": retriever | format_docs, "query": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

# Query the RAG Chain
rag_chain.invoke(
  query
)

'The instructor for the **AIM-5011-1: Natural Language Processing, Fall 2025** course at Katz School of Science and Health, Yeshiva University, is **Adam Faulkner (adam.faulkner@yu.edu)**.'

But our RAG system is fairly brittle: the system is unable to answer any questions that doesn't involve the provided context

In [ ]:
rag_chain.invoke(
  "When was America discovered?"
)

"I do not know the answer to your question based on the context provided. I'm sorry."

To solve this, we need to use the Agent paradigm. Specifically, we need a Planning component that allows the Agent to access its base weights when asked general questions but should access the RAG database when asked questions related to the class.